In [1]:
#Interview Practice
# SMS Spam Classifier



In [83]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split

#1. Analyze data
# load csv into numpy

csv = pd.read_csv("spam.csv",encoding='latin-1')
label = csv.iloc[:,0]
sms = csv.iloc[:,1]
csv.head()

# first look at the proportion of spam and ham
spam=0
ham=0

for row in label:
    if row=='ham':
        ham+=1
    else:
        spam+=1

spam_percent= spam/(spam+ham)
ham_percent = ham/(spam+ham)


#spam (0.13) is far less than ham (0.86)

0.13406317300789664

In [142]:
import re
from sklearn.feature_extraction import DictVectorizer
#2. Feature extraction/ preprocess 
# note: I realize there's some miniscule amount of text in coulmn 3,4,5, ignored here for simplicity


# tokenization, convert to lowercase, remove characters: .,*
def tokenization(spam):
    tokens={}
    spam= spam.strip()
    for token in re.split(r'[^A-Za-z0-9]+', spam):
        token=token.lower()
        if token not in tokens:
            tokens[token]= 1
        else:
            tokens[token]+=1
    return tokens

word_matrix=[]
for i in sms:
    word_matrix.append(tokenization(i))

# convert to sparse matrix
v = DictVectorizer(sparse=False)
X = v.fit_transform(word_matrix)
    
#split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train[0]


array([1., 0., 0., ..., 0., 0., 0.])

In [3]:
#3 Model selection
# Naive bayes is commonly used for text classification
# here just comparing 2 methods: multinomial and bernoulli

In [143]:
#4 Model training
# not sure about this, can use Sklearn or need to build the wheel myself?

from sklearn.naive_bayes import MultinomialNB,BernoulliNB

Mnb = MultinomialNB(alpha=1)
Mnb.fit(X, y)

Bnb = BernoulliNB()
Bnb.fit(X,y)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [144]:
#5 Model Evaluation

multinomial_accuracy= Mnb.score(X_test, y_test)
bernoulli_accuracy= Bnb.score(X_test, y_test)

print('multinomial accuracy: ', multinomial_accuracy)
print('bernoulli accuracy: ', bernoulli_accuracy)

#since multinomial gives higher accuracy rate, we go for multinomial

multinomial accuracy:  0.9928251121076234
bernoulli accuracy:  0.9883408071748879


In [152]:
#6 Prediction

ham_feed= "have a party tonight to celebrate winning the lottery, don't be late! Max"
spam_feed= "congratulation! you won the US Mega Pot, claim it before august 21 by replying to 042346742"


predict_ham=v.transform([tokenization(ham_feed)])  
predict_spam= v.transform([tokenization(spam_feed)])  


print('multinomial prediction for ham: ', Mnb.predict(predict_ham))
print('multinomial prediction for spam: ',Mnb.predict(predict_spam))

print('bernoulli prediction for ham: ',Bnb.predict(predict_ham))
print('bernoulli prediction for spam: ',Bnb.predict(predict_spam))

# multinomial predicts both case correct, bernoulli only 1, multinomial more accurate

multinomial prediction for ham:  ['ham']
multinomial prediction for spam:  ['spam']
bernoulli prediction for ham:  ['ham']
bernoulli prediction for spam:  ['ham']


In [ ]:
#7 not sure about this, do they expect parameter tuning to build some graph? 